<a href="https://colab.research.google.com/github/MCanela-1954/Data/blob/main/Notebooks/%5BRAG-9.1%5D%20QA%20on%20IESE%20Business%20School.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [RAG-9.1] Q/A on IESE Business School

## Setup

In [ ]:
from google import genai
from google.genai.types import GenerateContentConfig, EmbedContentConfig
import numpy as np, pandas as pd

In [ ]:
client = genai.Client(api_key = 'GOOGLE_API_KEY')

## Model selection

In [ ]:
chat_model = 'gemini-2.0-flash'

In [ ]:
embed_model = 'models/text-embedding-004'

In [ ]:
embed_config = EmbedContentConfig(task_type = 'SEMANTIC_SIMILARITY', output_dimensionality = 768)

## Importing the QA data

In [ ]:
path = 'https://raw.githubusercontent.com/IAlegre-MCanela/Summer_Course/main/Session%209/'

In [ ]:
data = pd.read_json(path + 'faq.json')
data.info()

## Importing the vector database

In [ ]:
embeds = pd.read_csv(path + 'embeds.csv', header = None)

## Encoding the query

In [ ]:
query = 'Are the EMBA courses taught in Spanish?'

In [ ]:
result = client.models.embed_content(config = embed_config, model = embed_model, contents = [query])

In [ ]:
query_embed = result.embeddings[0].values
query_embed = np.array(query_embed)
query_embed.shape

## Searching for the closest questions

In [ ]:
data['similarity'] = np.dot(embeds, query_embed.T)
print(data.head())

In [ ]:
top3 = data.sort_values(by='similarity', ascending=False).head(3)
print(top3)

## System guidelines

In [ ]:
guidelines = '''
You are a chatbot responding queries about MBA programs at IESE Business School.
You will be shown the query.
Make your response shorter than 50 words.
Use only the information provided.
If the information provided doesn't mention the subject of the query, limit your response to the following:
<response>
Unfortunately, I don't have information about this subject. Please, directly contact with the school.
</response>
'''

## Submitting the query

In [ ]:
info = 'Information:\n' + top3['answer'].sum()
print(info)

In [ ]:
chat_config = GenerateContentConfig(system_instruction = guidelines + info)

In [ ]:
response = client.models.generate_content(config = chat_config, contents = query, model = chat_model)
print(response.text)